In [1]:
import numpy as np, scipy.stats as st

import src
import src.tours as tours
from src.cross_entropy import CrossEntropy


In [10]:
# drone tour parameters
num_sites, num_drones = 10, 4
sites = np.random.rand(num_sites, 2)  # the 2 indicates we are sampling sites from the plane

drone_tour = tours.DroneTour(sites, num_drones)

# cross entropy parameters
x_entropy_sample_size = 5 # sample size
x_entropy_quantile = 0.2  # specifies the 90% percentile

x_entropy_estimator = CrossEntropy(drone_tour, x_entropy_sample_size, x_entropy_quantile)

# an example way to use the callback -- see below for details
sample_scores = []
hundreth_tour_costs = []

In [7]:
def callback(**kwargs):
    """

    :param kwargs:
    The keyword arguments passed to callback are
            - iteration, the current iteration
            - sample_states, the current sample of states in the given round of annealing
            - sample_scores, the scores associated with the sampled states in the given round of annealing
            - threshold, the scores which form the (1-quantile) quantile of the sample scores
            - distribution_parameters, the current parameters of the distribution of states in the state space
    :return:
    """
    sample_scores.append(kwargs['sample_scores'])  # for example, maybe we want a list of all sample scores during
    # the run of cross entropy

In [13]:
hundreth_tour_costs = []
for i in range(0,100):
    sample_scores = []
    x_entropy_estimator.minimize(callback=callback)
    hundreth_tour_costs.append(min(sample_scores[99]))
    
print(hundreth_tour_costs)
#print(len(sample_scores))

[2.5113449455241312, 1.9311228398076326, 1.7465253485980547, 2.0167493386745075, 1.7455019302313026, 2.024112894338163, 2.4640445230780474, 2.0631805868797817, 2.4896254592039404, 2.1380652563101186, 1.698588211382055, 1.7839216606235309, 1.631015914634682, 1.7467483344315826, 1.868850461432354, 2.4455565675167557, 2.3286843407762774, 1.9079965816935254, 1.917173637022419, 1.9113815876219757, 2.0877650971417037, 1.638004847543574, 2.05367489031718, 2.522912335456118, 1.9607680564910548, 1.927904396874422, 1.946443929051453, 2.423558530599638, 1.927904396874422, 1.7720404192048347, 1.8041299696228519, 1.824211132966156, 2.3542043001612676, 1.8837881718934657, 1.9095875606026749, 2.0411741710969302, 1.970075568036398, 2.386434551545732, 1.8480433138319337, 1.8013430726631179, 2.1807354787301496, 1.9650589588960954, 2.0275794627244585, 2.365198431280554, 1.3262854046521828, 2.3970460337662387, 1.753595410801418, 2.378628091643023, 1.6308104975008733, 2.4583784226202043, 1.6308104975008733

In [12]:
st.t.interval(0.95, len(hundreth_tour_costs)-1, loc=np.mean(hundreth_tour_costs), scale=st.sem(hundreth_tour_costs))

(1.9070079962512199, 2.3190466282779156)

In [14]:
len(hundreth_tour_costs)

110